In [1]:
import os
import subprocess
import sys
from pathlib import Path
import time

def run_notebook(notebook_path, output_dir=None):
    """
    Execute a Jupyter notebook using nbconvert.
    
    Args:
        notebook_path: Path to the notebook file
        output_dir: Directory to save output notebooks (optional)
    
    Returns:
        Tuple of (notebook_name, success_status, error_message)
    """
    notebook_name = Path(notebook_path).stem
    
    try:
        # Build command with proper list format for Windows paths with spaces
        cmd = [
            "jupyter", "nbconvert", 
            "--to", "notebook", 
            "--execute",
            "--ExecutePreprocessor.timeout=600"
        ]
        
        if output_dir:
            cmd.extend(["--output-dir", output_dir])
        
        cmd.append(notebook_path)
        
        print(f"▶️  Running: {notebook_name}")
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"✅ Completed: {notebook_name}")
            return (notebook_name, True, None)
        else:
            error_msg = result.stderr or result.stdout
            print(f"❌ Failed: {notebook_name}\n{error_msg}")
            return (notebook_name, False, error_msg)
    
    except Exception as e:
        print(f"❌ Error executing {notebook_name}: {str(e)}")
        return (notebook_name, False, str(e))


def main():
    # Configuration
    notebook_folder = r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Data Engineering\Model_Monitoring\Gini Monitoring\Gini_Monitoring_Modular_Approach\Notebooks"  # Change to your notebook folder path
    master_notebook = "CIC_SIL_Models.ipynb"
    output_folder = "./outputs"  # Optional: where to save executed notebooks
    
    # Create output folder if it doesn't exist
    if output_folder and not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get all notebooks in the folder
    all_notebooks = sorted(Path(notebook_folder).glob("*.ipynb"))
    
    if not all_notebooks:
        print("❌ No Jupyter notebooks found in the folder!")
        sys.exit(1)
    
    # Separate master notebook from others
    master_path = Path(notebook_folder) / master_notebook
    other_notebooks = [nb for nb in all_notebooks if nb.name != master_notebook]
    
    if not master_path.exists():
        print(f"❌ Master notebook '{master_notebook}' not found!")
        sys.exit(1)
    
    print(f"\n📋 Found {len(all_notebooks)} notebooks total")
    print(f"🎯 Master notebook: {master_notebook}")
    print(f"🔗 Other notebooks to run sequentially: {len(other_notebooks)}\n")
    
    # Track all results
    all_results = []
    total_start_time = time.time()
    
    # Step 1: Run master notebook first
    print("=" * 60)
    print("STEP 1: Running Master Notebook")
    print("=" * 60)
    start_time = time.time()
    
    master_result = run_notebook(str(master_path), output_folder)
    all_results.append(master_result)
    
    master_elapsed = time.time() - start_time
    print(f"⏱️  Master notebook took {master_elapsed:.2f} seconds\n")
    
    if not master_result[1]:
        print("❌ Master notebook failed! Exiting...")
        sys.exit(1)
    
    # Step 2: Run other notebooks sequentially
    print("=" * 60)
    print("STEP 2: Running Other Notebooks Sequentially")
    print("=" * 60)
    
    if not other_notebooks:
        print("ℹ️  No other notebooks to run.")
    else:
        for idx, notebook in enumerate(other_notebooks, 1):
            print(f"\n[{idx}/{len(other_notebooks)}]")
            start_time = time.time()
            
            result = run_notebook(str(notebook), output_folder)
            all_results.append(result)
            
            elapsed = time.time() - start_time
            print(f"⏱️  Took {elapsed:.2f} seconds\n")
            
            if not result[1]:
                print(f"⚠️  Notebook {notebook.name} failed. Continuing with next notebook...\n")
    
    # Summary
    total_elapsed = time.time() - total_start_time
    successful = sum(1 for _, success, _ in all_results if success)
    failed = len(all_results) - successful
    
    print("=" * 60)
    print("SUMMARY")
    print("=" * 60)
    print(f"✅ Successful: {successful}/{len(all_results)}")
    print(f"❌ Failed: {failed}/{len(all_results)}")
    
    if failed > 0:
        print("\nFailed notebooks:")
        for name, success, error in all_results:
            if not success:
                print(f"  - {name}: {error}")
    
    print(f"\n📊 Total execution time: {total_elapsed:.2f} seconds")


if __name__ == "__main__":
    main()


📋 Found 14 notebooks total
🎯 Master notebook: CIC_SIL_Models.ipynb
🔗 Other notebooks to run sequentially: 13

STEP 1: Running Master Notebook
▶️  Running: CIC_SIL_Models
✅ Completed: CIC_SIL_Models
⏱️  Master notebook took 2160.64 seconds

STEP 2: Running Other Notebooks Sequentially

[1/13]
▶️  Running: Alpha-Cash-CIC-Model
✅ Completed: Alpha-Cash-CIC-Model
⏱️  Took 285.37 seconds


[2/13]
▶️  Running: Alpha-Cash-Stack-Model
✅ Completed: Alpha-Cash-Stack-Model
⏱️  Took 266.63 seconds


[3/13]
▶️  Running: Alpha_Cash_Stack_Model_Credo_Score
✅ Completed: Alpha_Cash_Stack_Model_Credo_Score
⏱️  Took 273.31 seconds


[4/13]
▶️  Running: alpha_stack_model_sil
❌ Failed: alpha_stack_model_sil
[NbConvertApp] Converting notebook D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Data Engineering\Model_Monitoring\Gini Monitoring\Gini_Monitoring_Modular_Approach\Notebooks\alpha_stack_model_sil.ipynb to notebook
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - m